In [ ]:
import sys


import matplotlib.pyplot as plt
import mbuild
import nglview
import mdtraj

from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Draw

import openmm
from openff import toolkit, utilities, interchange, models
from openff.units import unit
from openff.interchange.components._packmol import UNIT_CUBE, pack_box

In [ ]:
molecules_rdkit = dict(
    ocm2=Chem.MolFromSmiles('CC(C(OCCOC(OCCOCCOC(OCCOC(C(C)=C)=O)=O)=O)=O)=C'),
    dmag=Chem.MolFromSmiles('C=C(C)C(OCCOC(C(C)=C)=O)=O'),
    peta=Chem.MolFromSmiles('C=CC(OCC(COC(C=C)=O)(COC(C=C)=O)CO)=O'),
    butanol1=Chem.MolFromSmiles('OCCCC'),
)


In [ ]:
Draw.MolToImage(molecules_rdkit['peta'])

In [ ]:
molecules = {}
for key, rdkit_mol in molecules_rdkit.items():
    mol=toolkit.Molecule.from_rdkit(rdkit_mol)
    molecules[key]=mol
    mol.generate_conformers(n_conformers=1)
    mol.name=key

# Parametrize

In [ ]:
# inters = {}
# for key in tqdm(iterable=molecules):

#     mol = molecules[key]
#     mol.assign_partial_charges(
#         partial_charge_method="am1bcc",
#         use_conformers=mol.conformers,
#     )
#     top = mol.to_topology()
#     ff = toolkit.ForceField('openff_unconstrained-2.1.0.offxml')
#     inter = interchange.Interchange.from_smirnoff(
#         force_field=ff,
#         topology=top,
#         charge_from_molecules=[mol],
#         partial_bond_orders_from_molecules=[mol],
#         allow_nonintegral_charges=True,
#     )
#     inters[key] = inter
# topology = {key: mol.to_topology() for key, mol in molecules.items()}

# Pack

In [ ]:
a = molecules['ocm2']
a.visualize(backend='nglview')

In [ ]:
box=pack_box(
    molecules=[molecules['ocm2'],molecules['butanol1']],
    number_of_copies=2 * [50],
    mass_density=850 * unit.kilogram / unit.meter**3,
    box_shape=UNIT_CUBE
)

In [ ]:
box.visualize()

In [ ]:
ff = toolkit.ForceField('openff_unconstrained-2.1.0.offxml')
box_parametrized= interchange.Interchange.from_smirnoff(
    force_field=ff,
    topology=box
)

In [ ]:
box_parametrized.minimize()

In [ ]:
# Logging options.
trj_freq = 10  # number of steps per written trajectory frame
data_freq = 10  # number of steps per written simulation statistics

# Integration options
time_step = 2 * openmm.unit.femtoseconds  # simulation timestep
temperature = 300 * openmm.unit.kelvin  # simulation temperature
friction = 1 / openmm.unit.picosecond  # friction constant

integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

In [ ]:
sim = box_parametrized.to_openmm_simulation(integrator=integrator)
sim.context.setVelocitiesToTemperature(temperature)
reporters = [
    openmm.app.StateDataReporter(sys.stdout, trj_freq, step=True),
    openmm.app.DCDReporter(
        'traj.dcd',
        trj_freq,  # enforcePeriodicBox=False,
    ),
    openmm.app.PDBReporter('solve2.pdb', trj_freq),
]
sim.reporters.extend(reporters)

In [ ]:
sim.step(1000)

In [ ]:
nglview.show_mdtraj(mdtraj.load('/home/base/WSL_base/Find_D/solve2.pdb'))